# V1

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Baca data
df = pd.read_csv('/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

# 🔑 Encode label pake if-else dari kolom 'categories'
def encode_label(cat):
    if cat == "inhibitor":
        return 1
    elif cat == "neutral":
        return 0
    else:
        raise ValueError(f"Label tidak dikenal: {cat}")

df['target'] = df['categories'].apply(encode_label)

# Cek distribusi
print("Distribusi label:")
print(df['target'].value_counts())
# 1 = inhibitor
# 0 = neutral

2025-10-18 18:53:16.758429: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-18 18:53:17.171720: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-18 18:53:18.957853: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Distribusi label:
target
0    321
1    315
Name: count, dtype: int64


In [2]:
# Split DULU! (stratify biar proporsi 0/1 seimbang)
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['target']  # ← wajib!
)

y_train = df_train['target'].values.astype(np.float32)
y_test = df_test['target'].values.astype(np.float32)

In [3]:
# Bikin vocab dari TRAIN doang
all_chars = set(''.join(df_train['smiles']))
charset = ['!', 'E', '?'] + sorted(all_chars)
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
max_len = max(len(s) for s in df_train['smiles']) + 3

def encode_smiles(smiles_list):
    sequences = []
    for s in smiles_list:
        seq = [char_to_int['!']]
        for c in s:
            seq.append(char_to_int.get(c, char_to_int['?']))
        seq.append(char_to_int['E'])
        if len(seq) < max_len:
            seq += [char_to_int['?']] * (max_len - len(seq))
        else:
            seq = seq[:max_len]
        sequences.append(seq)
    return np.array(sequences, dtype=np.int32)

X_train = encode_smiles(df_train['smiles'])
X_test = encode_smiles(df_test['smiles'])

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization

inputs = Input(shape=(max_len,))
x = Embedding(vocab_size, 64)(inputs)
x = Conv1D(192, 10, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv1D(192, 5, activation='relu', padding='same')(x)
x = Conv1D(192, 3, activation='relu', padding='same')(x)
x = GlobalMaxPooling1D()(x)
embedding = Dense(128, activation='relu')(x)

encoder = Model(inputs=inputs, outputs=embedding)

# Ekstrak embedding
train_emb = encoder.predict(X_train, verbose=1)
test_emb = encoder.predict(X_test, verbose=1)

2025-10-18 18:53:30.774905: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [5]:
# Reshape buat LSTM: (n, 1, 128)
X_train_lstm = train_emb.reshape(train_emb.shape[0], 1, 128)
X_test_lstm = test_emb.reshape(test_emb.shape[0], 1, 128)

# Model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(1, 128)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # ← sigmoid buat binary
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train
history = model.fit(
    X_train_lstm, y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_test_lstm, y_test),
    verbose=1
)

Epoch 1/50


/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.4921 - loss: 0.6932 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5039 - loss: 0.6932 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5039 - loss: 0.6933 - val_accuracy: 0.5078 - val_loss: 0.6929
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5039 - loss: 0.6930 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5039 - loss: 0.6931 - val_accuracy: 0.5781 - val_loss: 0.6930
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5059 - loss: 0.6930 - val_accuracy: 0.5078 - val_loss: 0.6929
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5039 - loss: 0.6929 - val_accuracy: 0.5078 - val_loss: 0.6928
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5039 - loss: 0.6930 - val_accuracy: 0.5078 - val_loss: 0.6927
Ep

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

tf.random.set_seed(42)

# Hyperparameters (Model M5)
units = [128, 64, 32]
dropout_rate = 0.6
learning_rate = 0.0005
batch_size = 32
l2_reg = 1e-4

# ✅ Pastikan X_train adalah embedding dari SMILES2Vec → shape (n, 128)
print("X_train shape:", X_train.shape)  # Harus (n_samples, 128)

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))  # ← (128,)

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM Layers
model_baseline_5.add(
    LSTM(
        units[0],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=True,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(
    LSTM(
        units[1],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=True,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(
    LSTM(
        units[2],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=False,
        kernel_regularizer=l2(l2_reg)
    )
)

model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

# Compile
model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

model_baseline_5.summary()

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),  # ← lebih akurat
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# Evaluasi
loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

X_train shape: (508, 166)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_4 (Reshape)             │ (None, 1, 166)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 1, 128)         │       151,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 1, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,897 (831.63 KB)

 Trainable params: 212,897 (831.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - AUC: 0.5175 - accuracy: 0.4941 - loss: 0.7411 - val_AUC: 0.6998 - val_accuracy: 0.6797 - val_loss: 0.7356
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.6233 - accuracy: 0.5925 - loss: 0.7329 - val_AUC: 0.7427 - val_accuracy: 0.7500 - val_loss: 0.7278
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.6832 - accuracy: 0.6457 - loss: 0.7245 - val_AUC: 0.7758 - val_accuracy: 0.6797 - val_loss: 0.7193
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.7600 - accuracy: 0.6949 - loss: 0.7126 - val_AUC: 0.7923 - val_accuracy: 0.7422 - val_loss: 0.7055
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.7827 - accuracy: 0.7205 - loss: 0.6943 - val_AUC: 0.7921 - val_accuracy: 0.7266 - val_loss: 0.6860
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.8263 - accuracy: 0.7579 - loss: 0.6602 - val_AUC: 0.8109 - val_accuracy: 0.7266 - val_loss: 0.6503
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

tf.random.set_seed(42)

# Hyperparameters (Model M5)
units = [64, 32]
dropout_rate = 0.6
learning_rate = 0.0001
batch_size = 32
l2_reg = 1e-2

# ✅ Pastikan X_train adalah embedding dari SMILES2Vec → shape (n, 128)
print("X_train shape:", X_train.shape)  # Harus (n_samples, 128)

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))  # ← (128,)

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM Layers
model_baseline_5.add(
    LSTM(
        units[0],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=True,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(
    LSTM(
        units[1],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=False,
        kernel_regularizer=l2(l2_reg)
    )
)


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

model_baseline_5.summary()

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),  # ← lebih akurat
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# Evaluasi
loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

X_train shape: (508, 166)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_9 (Reshape)             │ (None, 1, 166)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, 1, 64)          │        59,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,585 (279.63 KB)

 Trainable params: 71,585 (279.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - AUC: 0.5015 - accuracy: 0.5020 - loss: 3.4960 - val_AUC: 0.5252 - val_accuracy: 0.5000 - val_loss: 3.4450
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5140 - accuracy: 0.5217 - loss: 3.4068 - val_AUC: 0.5458 - val_accuracy: 0.5234 - val_loss: 3.3571
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5392 - accuracy: 0.5374 - loss: 3.3181 - val_AUC: 0.5695 - val_accuracy: 0.5469 - val_loss: 3.2720
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - AUC: 0.5254 - accuracy: 0.5118 - loss: 3.2367 - val_AUC: 0.6018 - val_accuracy: 0.5312 - val_loss: 3.1897
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.6116 - accuracy: 0.5886 - loss: 3.1496 - val_AUC: 0.6126 - val_accuracy: 0.5625 - val_loss: 3.1100
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5857 - accuracy: 0.5531 - loss: 3.0726 - val_AUC: 0.6233 - val_accuracy: 0.5938 - val_loss: 3.0337
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

tf.random.set_seed(42)

# Hyperparameters (Model M5)
units = [128, 32]
dropout_rate = 0.5
learning_rate = 0.0001
batch_size = 16
l2_reg = 1e-4

# ✅ Pastikan X_train adalah embedding dari SMILES2Vec → shape (n, 128)
print("X_train shape:", X_train.shape)  # Harus (n_samples, 128)

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))  # ← (128,)

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM Layers
model_baseline_5.add(
    LSTM(
        units[0],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=False,
        kernel_regularizer=l2(l2_reg)
    )
)


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

model_baseline_5.summary()

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),  # ← lebih akurat
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# Evaluasi
loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

X_train shape: (508, 166)


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_13 (Reshape)            │ (None, 1, 166)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ (None, 128)            │       151,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,169 (590.50 KB)

 Trainable params: 151,169 (590.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - AUC: 0.5290 - accuracy: 0.5197 - loss: 0.7520 - val_AUC: 0.6093 - val_accuracy: 0.5781 - val_loss: 0.7041
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5456 - accuracy: 0.5256 - loss: 0.7296 - val_AUC: 0.6618 - val_accuracy: 0.6406 - val_loss: 0.6907
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6109 - accuracy: 0.5709 - loss: 0.6944 - val_AUC: 0.6819 - val_accuracy: 0.6406 - val_loss: 0.6850
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6358 - accuracy: 0.6024 - loss: 0.6846 - val_AUC: 0.7138 - val_accuracy: 0.6641 - val_loss: 0.6738
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6631 - accuracy: 0.5984 - loss: 0.6731 - val_AUC: 0.7380 - val_accuracy: 0.6953 - val_loss: 0.6671
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6545 - accuracy: 0.6063 - loss: 0.6796 - val_AUC: 0.7244 - val_accuracy: 0.6875 - val_loss: 0.6647
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms

# V2

In [20]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, 
    BatchNormalization, Dropout, Lambda, LSTM
)
from tensorflow.keras import backend as K

In [21]:
# ============================================================
# === 1. Vocabulary & Encoding
# ============================================================

# Bikin vocab dari TRAIN doang
all_chars = set(''.join(df_train['smiles']))
charset = ['!', 'E', '?', '_'] + sorted(all_chars)  # pad token '_'
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
max_len = max(len(s) for s in df_train['smiles']) + 2  # start + end token

def encode_smiles(smiles_list):
    sequences = []
    for s in smiles_list:
        seq = [char_to_int['!']]  # start
        for c in s:
            seq.append(char_to_int.get(c, char_to_int['?']))
        seq.append(char_to_int['E'])  # end
        # padding
        if len(seq) < max_len:
            seq += [char_to_int['_']] * (max_len - len(seq))
        else:
            seq = seq[:max_len]
        sequences.append(seq)
    return np.array(sequences, dtype=np.int32)

X_train = encode_smiles(df_train['smiles'])
X_test = encode_smiles(df_test['smiles'])

In [22]:
# ============================================================
# === 2. Encoder (SMILES2Vec)
# ============================================================

inputs = Input(shape=(max_len,))
x = Embedding(vocab_size, 64)(inputs)
x = Conv1D(192, 10, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv1D(192, 5, activation='relu', padding='same')(x)
x = Conv1D(192, 3, activation='relu', padding='same')(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.3)(x)
embedding = Dense(128, activation='relu')(x)
embedding = Lambda(lambda y: K.l2_normalize(y, axis=1))(embedding)  

encoder = Model(inputs=inputs, outputs=embedding)
encoder.summary()

Model: "functional_69"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 165)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 165, 64)        │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 165, 192)       │       123,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 165, 192)       │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 165, 192)       │       184,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 165, 192)       │       110,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 192)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 128)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 445,824 (1.70 MB)

 Trainable params: 445,440 (1.70 MB)

 Non-trainable params: 384 (1.50 KB)

In [23]:
# ============================================================
# === 3. Ekstrak Embedding
# ============================================================

train_emb = encoder.predict(X_train, verbose=1)
test_emb = encoder.predict(X_test, verbose=1)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [24]:
# ============================================================
# === 4. Training LSTM Classifier
# ============================================================

# Reshape buat LSTM: (n, 1, 128)
X_train_lstm = train_emb.reshape(train_emb.shape[0], 1, 128)
X_test_lstm = test_emb.reshape(test_emb.shape[0], 1, 128)

model = Sequential([
    LSTM(64, input_shape=(1, 128)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # sigmoid buat binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train_lstm, y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_test_lstm, y_test),
    verbose=1
)

Epoch 1/50


/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5118 - loss: 0.6929 - val_accuracy: 0.5078 - val_loss: 0.6926
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5079 - loss: 0.6936 - val_accuracy: 0.5078 - val_loss: 0.6924
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5295 - loss: 0.6925 - val_accuracy: 0.5078 - val_loss: 0.6922
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4941 - loss: 0.6933 - val_accuracy: 0.5156 - val_loss: 0.6921
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4980 - loss: 0.6933 - val_accuracy: 0.5547 - val_loss: 0.6918
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5059 - loss: 0.6929 - val_accuracy: 0.5469 - val_loss: 0.6913
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5236 - loss: 0.6919 - val_accuracy: 0.5078 - val_loss: 0.6910
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5236 - loss: 0.6914 - val_accuracy: 0.5078 - val_loss: 0.6904
Ep

In [36]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [256, 32]
dropout_rate = 0.5
learning_rate = 0.0001
batch_size = 16
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(
    tf.keras.layers.LSTM(
        units[0],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=False,
        kernel_regularizer=l2(l2_reg)
    )
)

model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_25 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_50 (LSTM)                  │ (None, 256)            │       432,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 432,385 (1.65 MB)

 Trainable params: 432,385 (1.65 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5197 - auc_25: 0.5208 - loss: 0.7538 - val_accuracy: 0.6016 - val_auc_25: 0.6040 - val_loss: 0.7032
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5925 - auc_25: 0.6215 - loss: 0.6986 - val_accuracy: 0.6172 - val_auc_25: 0.6811 - val_loss: 0.6741
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6476 - auc_25: 0.7116 - loss: 0.6522 - val_accuracy: 0.6328 - val_auc_25: 0.7106 - val_loss: 0.6602
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6850 - auc_25: 0.7590 - loss: 0.6237 - val_accuracy: 0.6719 - val_auc_25: 0.7254 - val_loss: 0.6464
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6988 - auc_25: 0.7760 - loss: 0.6101 - val_accuracy: 0.6719 - val_auc_25: 0.7414 - val_loss: 0.6384
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7028 - auc_25: 0.7760 - loss: 0.6021 - val_accuracy: 0.6797 - val_auc_25: 0.7534 - val_loss: 0.6275
Epoch 7/1

In [35]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [128, 64]
dropout_rate = 0.5
learning_rate = 0.0001
batch_size = 16
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[1], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_24 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_48 (LSTM)                  │ (None, 1, 128)         │       150,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_49 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,001 (781.25 KB)

 Trainable params: 200,001 (781.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5197 - auc_24: 0.5425 - loss: 0.7312 - val_accuracy: 0.5781 - val_auc_24: 0.6106 - val_loss: 0.7263
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5374 - auc_24: 0.5714 - loss: 0.7273 - val_accuracy: 0.5703 - val_auc_24: 0.6447 - val_loss: 0.7219
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6122 - auc_24: 0.6437 - loss: 0.7179 - val_accuracy: 0.5781 - val_auc_24: 0.6657 - val_loss: 0.7179
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6260 - auc_24: 0.6881 - loss: 0.7091 - val_accuracy: 0.6172 - val_auc_24: 0.6780 - val_loss: 0.7142
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6181 - auc_24: 0.6743 - loss: 0.7077 - val_accuracy: 0.6172 - val_auc_24: 0.6761 - val_loss: 0.7110
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7028 - auc_24: 0.7540 - loss: 0.6947 - val_accuracy: 0.6328 - val_auc_24: 0.6784 - val_loss: 0.7066
Epoch 7/1

In [34]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [128, 64, 32]
dropout_rate = 0.5
learning_rate = 0.0001
batch_size = 16
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[1], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[2], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_23 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_45 (LSTM)                  │ (None, 1, 128)         │       150,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_46 (LSTM)                  │ (None, 1, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_47 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,385 (829.63 KB)

 Trainable params: 212,385 (829.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5118 - auc_23: 0.5140 - loss: 0.7422 - val_accuracy: 0.4766 - val_auc_23: 0.5175 - val_loss: 0.7410
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5827 - auc_23: 0.6155 - loss: 0.7378 - val_accuracy: 0.5391 - val_auc_23: 0.5742 - val_loss: 0.7378
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6280 - auc_23: 0.6587 - loss: 0.7339 - val_accuracy: 0.5312 - val_auc_23: 0.5984 - val_loss: 0.7351
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6220 - auc_23: 0.6826 - loss: 0.7309 - val_accuracy: 0.5938 - val_auc_23: 0.6230 - val_loss: 0.7321
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6713 - auc_23: 0.7339 - loss: 0.7262 - val_accuracy: 0.5859 - val_auc_23: 0.6698 - val_loss: 0.7288
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7087 - auc_23: 0.7586 - loss: 0.7219 - val_accuracy: 0.6172 - val_auc_23: 0.6781 - val_loss: 0.7253
Epoch 7/1

| Model  | LSTM Layers | Units (tiap layer) | Dropout | Learning Rate | Batch Size | L2 Reg | Epochs | Keterangan                            |
| :----- | :---------- | :----------------- | :------ | :------------ | :--------- | :----- | :----- | :------------------------------------ |
| **M1** | 1           | [64]               | 0.3     | 0.001         | 32         | 1e-4   | 50     | Baseline ringan, 1 layer kecil        |
| **M2** | 1           | [128]              | 0.5     | 0.0005        | 32         | 1e-4   | 75     | Mirip model lo sekarang               |
| **M3** | 2           | [128, 64]          | 0.5     | 0.0001        | 16         | 1e-4   | 100    | Deep LSTM, lebih stabil               |
| **M4** | 3           | [256, 128, 64]     | 0.4     | 0.0001        | 16         | 5e-5   | 100    | Multi-layer untuk kompleksitas tinggi |
| **M5** | 2           | [128, 32]          | 0.5     | 0.0001        | 16         | 1e-4   | 100    | Model yang lagi lo pakai sekarang     |


In [38]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [64, 64, 32]
dropout_rate = 0.3
learning_rate = 0.001
batch_size = 32
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_27 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_52 (LSTM)                  │ (None, 64)             │        58,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,945 (230.25 KB)

 Trainable params: 58,945 (230.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5295 - auc_27: 0.5432 - loss: 0.7286 - val_accuracy: 0.6172 - val_auc_27: 0.6551 - val_loss: 0.6791
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5728 - auc_27: 0.6033 - loss: 0.6913 - val_accuracy: 0.6484 - val_auc_27: 0.7029 - val_loss: 0.6588
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6181 - auc_27: 0.6852 - loss: 0.6548 - val_accuracy: 0.6172 - val_auc_27: 0.6875 - val_loss: 0.6569
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6437 - auc_27: 0.6839 - loss: 0.6520 - val_accuracy: 0.6562 - val_auc_27: 0.7216 - val_loss: 0.6434
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6909 - auc_27: 0.7366 - loss: 0.6263 - val_accuracy: 0.6250 - val_auc_27: 0.7049 - val_loss: 0.6416
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6772 - auc_27: 0.7509 - loss: 0.6115 - val_accuracy: 0.6875 - val_auc_27: 0.7324 - val_loss: 0.6264
Epoch 7/1

In [39]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [128, 64, 32]
dropout_rate = 0.5
learning_rate = 0.0005
batch_size = 32
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_28 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_53 (LSTM)                  │ (None, 128)            │       150,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 150,657 (588.50 KB)

 Trainable params: 150,657 (588.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.5177 - auc_28: 0.4983 - loss: 0.7738 - val_accuracy: 0.5938 - val_auc_28: 0.6834 - val_loss: 0.6786
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6063 - auc_28: 0.6320 - loss: 0.6860 - val_accuracy: 0.6016 - val_auc_28: 0.6802 - val_loss: 0.6753
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6476 - auc_28: 0.6783 - loss: 0.6661 - val_accuracy: 0.6562 - val_auc_28: 0.7160 - val_loss: 0.6569
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6358 - auc_28: 0.7064 - loss: 0.6484 - val_accuracy: 0.6641 - val_auc_28: 0.7657 - val_loss: 0.6344
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6831 - auc_28: 0.7458 - loss: 0.6242 - val_accuracy: 0.7031 - val_auc_28: 0.7770 - val_loss: 0.6258
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6850 - auc_28: 0.7435 - loss: 0.6257 - val_accuracy: 0.7109 - val_auc_28: 0.7637 - val_loss: 0.6283
Epoch 7/1

In [41]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [128, 64, 32]
dropout_rate = 0.5
learning_rate = 0.0001
batch_size = 16
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[1], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_30 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_56 (LSTM)                  │ (None, 1, 128)         │       150,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_57 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,001 (781.25 KB)

 Trainable params: 200,001 (781.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.4961 - auc_30: 0.4910 - loss: 0.7364 - val_accuracy: 0.6250 - val_auc_30: 0.6818 - val_loss: 0.7241
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5571 - auc_30: 0.5792 - loss: 0.7268 - val_accuracy: 0.6406 - val_auc_30: 0.7139 - val_loss: 0.7196
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5945 - auc_30: 0.6274 - loss: 0.7202 - val_accuracy: 0.6797 - val_auc_30: 0.7447 - val_loss: 0.7148
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6181 - auc_30: 0.6662 - loss: 0.7137 - val_accuracy: 0.7031 - val_auc_30: 0.7571 - val_loss: 0.7095
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6437 - auc_30: 0.7185 - loss: 0.7058 - val_accuracy: 0.7109 - val_auc_30: 0.7618 - val_loss: 0.7041
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6693 - auc_30: 0.7352 - loss: 0.6990 - val_accuracy: 0.6953 - val_auc_30: 0.7648 - val_loss: 0.6999
Epoch 7/1

In [44]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [256, 128, 64]
dropout_rate = 0.4
learning_rate = 0.0001
batch_size = 16
l2_reg = 5e-5

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[1], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[2], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_33 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_62 (LSTM)                  │ (None, 1, 256)         │       432,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_63 (LSTM)                  │ (None, 1, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_64 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 678,721 (2.59 MB)

 Trainable params: 678,721 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.5551 - auc_33: 0.5313 - loss: 0.7314 - val_accuracy: 0.6328 - val_auc_33: 0.6803 - val_loss: 0.7278
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6260 - auc_33: 0.6770 - loss: 0.7246 - val_accuracy: 0.6953 - val_auc_33: 0.7324 - val_loss: 0.7234
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6713 - auc_33: 0.7294 - loss: 0.7191 - val_accuracy: 0.7031 - val_auc_33: 0.7358 - val_loss: 0.7176
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6870 - auc_33: 0.7642 - loss: 0.7118 - val_accuracy: 0.7031 - val_auc_33: 0.7593 - val_loss: 0.7101
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7205 - auc_33: 0.7889 - loss: 0.7003 - val_accuracy: 0.7109 - val_auc_33: 0.7866 - val_loss: 0.6984
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7402 - auc_33: 0.8076 - loss: 0.6828 - val_accuracy: 0.7188 - val_auc_33: 0.7980 - val_loss: 0.6803
Epoc

In [45]:
# ============================================================
# 🔹 5. LSTM Baseline Model (M5)
# ============================================================

# Hyperparameters
units = [128, 32, 32]
dropout_rate = 0.5
learning_rate = 0.0001
batch_size = 16
l2_reg = 1e-4

# Build model
model_baseline_5 = Sequential()

# Input: terima flat vector (128,)
model_baseline_5.add(tf.keras.Input(shape=(X_train.shape[1],)))

# Reshape jadi (1, 128) → 1 timestep, 128 features
model_baseline_5.add(tf.keras.layers.Reshape((1, X_train.shape[1])))

# LSTM layer
model_baseline_5.add(tf.keras.layers.LSTM(units[0], activation='tanh', recurrent_activation='sigmoid', return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_baseline_5.add(tf.keras.layers.LSTM(units[1], activation='tanh', recurrent_activation='sigmoid', return_sequences=False, kernel_regularizer=l2(l2_reg)))


model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

model_baseline_5.summary()

# ============================================================
# 🔹 6. Train Model
# ============================================================

history_baseline_5 = model_baseline_5.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
    ],
    verbose=1
)

# ============================================================
# 🔹 7. Evaluation
# ============================================================

loss, acc, auc = model_baseline_5.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_34 (Reshape)            │ (None, 1, 165)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_65 (LSTM)                  │ (None, 1, 128)         │       150,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_66 (LSTM)                  │ (None, 32)             │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,169 (668.63 KB)

 Trainable params: 171,169 (668.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.5335 - auc_34: 0.5331 - loss: 0.7281 - val_accuracy: 0.5391 - val_auc_34: 0.5724 - val_loss: 0.7259
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5728 - auc_34: 0.6017 - loss: 0.7200 - val_accuracy: 0.5625 - val_auc_34: 0.5908 - val_loss: 0.7220
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6102 - auc_34: 0.6516 - loss: 0.7148 - val_accuracy: 0.5469 - val_auc_34: 0.6269 - val_loss: 0.7187
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6260 - auc_34: 0.6685 - loss: 0.7110 - val_accuracy: 0.5781 - val_auc_34: 0.6455 - val_loss: 0.7151
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6732 - auc_34: 0.7285 - loss: 0.7033 - val_accuracy: 0.5859 - val_auc_34: 0.6684 - val_loss: 0.7116
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6752 - auc_34: 0.7297 - loss: 0.6990 - val_accuracy: 0.6094 - val_auc_34: 0.6775 - val_loss: 0.7081
Epoch 7/1